# Converting Images to Point Cloud

In [ ]:
#@title 1. Load Libraries
import glob
import numpy as np
import os
import struct
import cv2
from tqdm.notebook import tqdm

In [ ]:
#@title 2. Additional functions for Point Cloud generation



def rotate(plane, angle):
    theta = np.radians(angle)
    # rot_x = np.array([[1, 0, 0], [0, np.cos(theta), -np.sin(theta)], [0, np.sin(theta), np.cos(theta)]])
    rot_y = np.array([[np.cos(theta), 0, np.sin(theta)], [0, 1, 0], [-np.sin(theta), 0, np.cos(theta)]])
    # rot_z = np.array([[np.cos(theta), -np.sin(theta), 0], [np.sin(theta), np.cos(theta), 0], [0, 0, 1]])
    return plane.dot(rot_y)


def img2points(img, d=0, a=0, thr=127):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # th = cv2.adaptiveThreshold(gray, thr, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 4)
    _, th = cv2.threshold(gray, thr, 255, cv2.THRESH_BINARY)
    depth = 1 - th / 255
    new_depth = depth + d
    w, h = new_depth.shape

    pixel_x, pixel_y = np.meshgrid(np.linspace(0, h - 1, h),np.linspace(0, w - 1, w))
    camera_points = np.zeros((np.size(pixel_x), 3))
    camera_points[:, 0] = np.reshape(pixel_x, -1)
    camera_points[:, 1] = np.reshape(pixel_y, -1)
    camera_points[:, 2] = np.reshape(new_depth, -1)

    color_points = img.reshape(-1, 3)
    if a % 360 != 0:
        camera_points = rotate(camera_points, a)

    valid_depth_ind = np.where(depth.flatten() > 0)[0]
    camera_points = camera_points[valid_depth_ind, :]
    color_points = color_points[valid_depth_ind, :]
    color_points = color_points.astype(int)

    return camera_points, color_points


def write_pointcloud(filename, xyz_points, rgb_points=None):
    """ creates a .ply file of the generated point clouds
    """

    assert xyz_points.shape[1] == 3, 'Input XYZ points should be Nx3 float array'
    if rgb_points is None:
        rgb_points = np.ones(xyz_points.shape).astype(np.uint8) * 255
    assert xyz_points.shape == rgb_points.shape, 'Input RGB colors should be Nx3 float array and have same size as input XYZ points'

    # Write header of .ply file
    with open(filename, 'wb') as fid:
        fid.write(bytes('ply\n', 'utf-8'))
        fid.write(bytes('format binary_little_endian 1.0\n', 'utf-8'))
        fid.write(bytes(f'element vertex {xyz_points.shape[0]}\n', 'utf-8'))
        fid.write(bytes('property float x\n', 'utf-8'))
        fid.write(bytes('property float y\n', 'utf-8'))
        fid.write(bytes('property float z\n', 'utf-8'))
        fid.write(bytes('property uchar red\n', 'utf-8'))
        fid.write(bytes('property uchar green\n', 'utf-8'))
        fid.write(bytes('property uchar blue\n', 'utf-8'))
        fid.write(bytes('end_header\n', 'utf-8'))

        # Write 3D points to .ply file
        for i in range(xyz_points.shape[0]):
            fid.write(bytearray(struct.pack("fffBBB", xyz_points[i, 0], xyz_points[i, 1], xyz_points[i, 2],
                                            rgb_points[i, 0], rgb_points[i, 1],
                                            rgb_points[i, 2])))


def sparse_image(img, level):
    new_image = np.zeros(img.shape, dtype=np.uint8)
    new_image.fill(255)
    new_image[::level,::level] = img[::level,::level]
    return new_image

In [ ]:
#@title 3. Mount Google Drive
#@markdown Access your Google Drive to upload your images and save the results.



from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title 4. Conversion of Images to Point Cloud

#@markdown Make sure you have loaded the libraries and mounted Google Drive before running this cell. You will be able to generate a 3D point cloud with the images you have generated as a result of training your model.

#@markdown Indicate the ordinal value of layers to skip: A high number will lower the density of the point cloud.
skip_layer =  5#@param {type: "integer"}
#@markdown Indicate the ordinal value of layers to skip: A high number will lower the density of the point cloud.
skip_points = 5 #@param {type: "integer"}
#@markdown Distance between the layers (if the value is greater than 1 the distance will be greater, if you choose a value between 0 and 1 the distance will be reduced)
distance = 1 #@param {type: "number"}
#@markdown Image path
image_dir = "" #@param {type: "string"}
#@markdown Path to save the Point Cloud
results_cloud = "" #@param {type: "string"}
#@markdown Define the name of the Point Cloud
cloud_name = "" #@param {type: "string"}


if image_dir=="":
  raise Exception("Please specify image directory")
if image_dir[-1] != "/":
    image_dir += "/"
if results_cloud=="":
  results_cloud= '/content/'
if results_cloud[-1] != "/":
    results_cloud += "/"
if not os.path.exists(results_cloud):
  os.makedirs(results_cloud)
if cloud_name=="":
  raise Exception("Please specify point cloud file name!")

img_files = sorted(glob.glob(f'{image_dir}*.png')+glob.glob(f'{image_dir}*.jpg'))
total = len(img_files)
assert total>0, 'Image directory is empty!'
print(f'{total} images loaded')
all_points = np.empty((0, 3))
all_colors = np.empty((0, 3))



# for i, fname in enumerate(img_files):
for i, fname in tqdm(enumerate(img_files), total=total):
    if i % skip_layer != 0:
        continue
    image = cv2.imread(fname, 1)
    # image = cv2.resize(image, (1024,1024), interpolation = cv2.INTER_LINEAR)
    image = sparse_image(image, skip_points)
    points, colors = img2points(image, d=i*distance, a=0, thr=125)
    all_points = np.concatenate((all_points, points))
    all_colors = np.concatenate((all_colors, colors))


write_pointcloud(f'{results_cloud}{cloud_name}.ply', all_points, all_colors.astype(np.uint8))
print(f'Cloud saved to {results_cloud}{cloud_name}.ply')